## Dirac equation

In [ ]:
import scipy 
from scipy import pi
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.style
import tensorflow as tf

In [ ]:
def reshaping(x):
    """
    Reshape the data created with numpy. 
    """
    return x.reshape(x.shape[0], 1)

In [ ]:
SMALL_SIZE = 20
MEDIUM_SIZE = 22
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
N0 = 1000
Nb = 1000 # Number of points in the boundaries and initial value.  
Nf = 20000 # Number of collocated points.

SMALL = 1e-06

In [ ]:
space_IV = np.random.uniform(0 + SMALL, 1 - SMALL, N0)
space_IV = reshaping(space_IV)
x_IV = space_IV
t_IV = 0*space_IV
u_IV_1 = x_IV**2 - x_IV + 1/6
v_IV_1 = 0.0*x_IV
u_IV_2 = np.piecewise(x_IV, [x_IV < 0.5, x_IV > 0.5], [lambda x_IV: x_IV - 0.25, lambda x_IV: -x_IV + 0.75])
v_IV_2 = 0.0*x_IV

In [ ]:
# Boundary Condition
time_BC = np.random.uniform(0 + SMALL, pi/2  - SMALL ,Nb)
time_BC = reshaping(time_BC)
# Boundary Condition x = 0
t_lb = time_BC
x_lb = 0*time_BC  
u_lb_1 = 0*x_lb
# Boundary Condition x = 1
t_ub = time_BC
x_ub = 0*time_BC + 1
u_ub_1 = 0*x_ub
# Boundary Condition
u_BC_1 = np.concatenate((u_lb_1, u_ub_1), axis=0)


In [ ]:
# Boundary Condition x = 0
v_lb_1 = 0*x_lb
# Boundary Condition x = 1
v_ub_1 = 0*x_ub
# Boundary Condition
v_BC_1 = np.concatenate((v_lb_1, v_ub_1), axis=0)

In [ ]:
# Boundary Condition x = 0
u_lb_2 = 0*x_lb
# Boundary Condition x = 1
u_ub_2 = 0*x_ub
# Boundary Condition
u_BC_2 = np.concatenate((u_lb_2, u_ub_2), axis=0)

In [ ]:
# Boundary Condition x = 0
v_lb_2 = 0*x_lb
# Boundary Condition x = 1
v_ub_2 = 0*x_ub
# Boundary Condition
v_BC_2 = np.concatenate((v_lb_2, v_ub_2), axis=0)
t_BC = np.concatenate((t_lb, t_ub), axis=0)
x_BC = np.concatenate((x_lb, x_ub), axis=0)

In [ ]:
x_collocated = np.random.uniform(0 + SMALL, 1 - SMALL, Nf)
x_collocated = reshaping(x_collocated)
t_collocated = np.random.uniform(0 + SMALL, pi/2 - SMALL, Nf)
t_collocated = reshaping(t_collocated)
xx, tt = np.concatenate((x_collocated, x_IV, x_BC), axis=0), np.concatenate((t_collocated, t_IV, t_BC), axis=0)
train_dataset = tf.data.Dataset.from_tensor_slices((xx, tt))
batch_size = xx.shape[0]
train_dataset = train_dataset.batch(batch_size)

In [ ]:
n_targets = 4
n_hidden_layers, n_hidden_neurons = 9,20
input_x = tf.keras.Input(shape=(1,))
input_t = tf.keras.Input(shape=(1,))
inputs = tf.keras.layers.concatenate([input_x, input_t])
initializer = tf.keras.initializers.GlorotNormal()
x = tf.keras.layers.Dense(n_hidden_neurons, activation='tanh', kernel_initializer=initializer, dtype=tf.float64)(inputs)
for hidden in range(n_hidden_layers-1):
    initializer = tf.keras.initializers.GlorotNormal()
    x = tf.keras.layers.Dense(n_hidden_neurons, activation='tanh', kernel_initializer=initializer, dtype=tf.float64)(x)
initializer = tf.keras.initializers.GlorotNormal()
outputs = tf.keras.layers.Dense(n_targets, activation='tanh', kernel_initializer=initializer, dtype=tf.float64)(x)

model = tf.keras.Model(inputs=[input_x, input_t], outputs=outputs)

In [ ]:
# Instantiate an optimizer to train the model.
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-02,
    decay_steps=100,
    decay_rate=0.96)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
loss_fn = tf.keras.losses.MeanSquaredError()

In [ ]:
@tf.function
def train_step(x, t):
    with tf.GradientTape() as tape:
        
        # Boundary conditions evaluation
        BC_pred = model([x_BC, t_BC], training = True)
        u_BC_1_pred = BC_pred[:, 0:1]
        v_BC_1_pred = BC_pred[:, 1:2]
        u_BC_2_pred = BC_pred[:, 2:3]
        v_BC_2_pred = BC_pred[:, 3:4]

        loss_u_BC_1 = loss_fn(u_BC_1, u_BC_1_pred)
        loss_v_BC_1 = loss_fn(v_BC_1, v_BC_1_pred)
        loss_u_BC_2 = loss_fn(u_BC_2, u_BC_2_pred)
        loss_v_BC_2 = loss_fn(v_BC_2, v_BC_2_pred)
 
        # Initial values evaluation
        IV_pred = model([x_IV, t_IV], training=True) 
        u_IV_1_pred = IV_pred[:, 0:1] 
        v_IV_1_pred = IV_pred[:, 1:2] 
        u_IV_2_pred = IV_pred[:, 2:3] 
        v_IV_2_pred = IV_pred[:, 3:4] 
        
        loss_u_IV_1 = 1000000*loss_fn(u_IV_1, u_IV_1_pred)
        loss_v_IV_1 = 1000000*loss_fn(v_IV_1, v_IV_1_pred)
        loss_u_IV_2 = 1000000*loss_fn(u_IV_2, u_IV_2_pred)
        loss_v_IV_2 = 1000000*loss_fn(v_IV_2, v_IV_2_pred)
        
        # Collocated points evaluation - Differential Equation 
        with tf.GradientTape(persistent=True) as tape_diff:
            tape_diff.watch(x)
            tape_diff.watch(t)
            pred = model([x, t], training=True)
            u_1_pred = pred[:, 0:1]
            v_1_pred = pred[:, 1:2]
            u_2_pred = pred[:, 2:3]
            v_2_pred = pred[:, 3:4]
            
            
            u_1_x  = tape_diff.gradient(u_1_pred, x)
            u_2_x  = tape_diff.gradient(u_2_pred, x)
            v_1_x = tape_diff.gradient(v_1_pred, x)
            v_2_x = tape_diff.gradient(v_2_pred, x)
            u_1_t = tape_diff.gradient(u_1_pred, t)
            v_1_t = tape_diff.gradient(v_1_pred, t)
            u_2_t = tape_diff.gradient(u_2_pred, t)
            v_2_t = tape_diff.gradient(v_2_pred, t)

        del tape_diff
        

        f_u_1 = u_1_t + u_2_x - v_1_pred
        f_v_1 = v_1_t + v_2_x + u_1_pred
        f_u_2 = -u_2_t - u_1_x - v_2_pred
        f_v_2 = -v_2_t - v_1_x + u_2_pred
    
        
        loss_f_u_1 = loss_fn(0, f_u_1)
        loss_f_v_1 = loss_fn(0, f_v_1)
        loss_f_u_2 = loss_fn(0, f_u_2)
        loss_f_v_2 = loss_fn(0, f_v_2)
        
        h_1_pred = u_1_pred**2 + v_1_pred**2
        h_2_pred = u_2_pred**2 + v_2_pred**2
        
        
        
        
        # General loss function
        loss = loss_f_u_1 + loss_f_v_1 + loss_f_u_2 + loss_f_v_2 + loss_u_BC_1 + loss_u_BC_2 + loss_v_BC_1 + loss_v_BC_2 + loss_u_IV_1 + loss_v_IV_1 + loss_u_IV_2 + loss_v_IV_2
        
    # Regular backpropagation
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
    
    return loss,loss_f_u_1 ,loss_f_v_1, loss_f_u_2, loss_f_v_2, loss_u_BC_1, loss_u_BC_2, loss_v_BC_1, loss_v_BC_2, loss_u_IV_1, loss_v_IV_1, loss_u_IV_2, loss_v_IV_2 

In [ ]:
def Callback_EarlyStopping(LossList, min_delta=0.1, patience=20):
    #No early stopping for 2*patience epochs 
    if len(LossList)//patience < 2 :
        return False
    #Mean loss for last patience epochs and second-last patience epochs
    mean_previous = np.mean(LossList[::-1][patience:2*patience]) #second-last
    mean_recent = np.mean(LossList[::-1][:patience]) #last
    #you can use relative or absolute change
    delta_abs = np.abs(mean_recent - mean_previous) #abs change
    delta_abs = np.abs(delta_abs / mean_previous)  # relative change
    if delta_abs < min_delta :
        print("*CB_ES* Loss didn't change much from last %d epochs"%(patience))
        print("*CB_ES* Percent change in loss value:", delta_abs*1e2)
        return True
    else:
        return False

In [ ]:
epochs = 500
start_time_outer = time.time()
loss_seq = []
loss_seq = []
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time_inner = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, t_batch_train) in enumerate(train_dataset):
        loss_train, loss_f_u_1, loss_f_v_1, loss_f_u_2, loss_f_v_2, loss_u_BC_1, loss_u_BC_2, loss_v_BC_1, loss_v_BC_2, loss_u_IV_1, loss_v_IV_1, loss_u_IV_2, loss_v_IV_2 = train_step(x_batch_train, t_batch_train)

        # Log every 200 batches.
        if step % 200 == 0:
            print(f"Training loss (for one batch) at step {step}: {format(float(loss_train), 'e')}")
            print(f"f_u_1 loss (for one batch) at step {step}: {format(float(loss_f_u_1), 'e')}")
            print(f"f_u_2 loss (for one batch) at step {step}: {format(float(loss_f_u_2), 'e')}")
            print(f"f_v_1 loss (for one batch) at step {step}: {format(float(loss_f_v_1), 'e')}")
            print(f"f_v_2 loss (for one batch) at step {step}: {format(float(loss_f_v_2), 'e')}")
            print(f"u_BC_1 loss (for one batch) at step {step}: {format(float(loss_u_BC_1), 'e')}")
            print(f"u_BC_2 loss (for one batch) at step {step}: {format(float(loss_u_BC_2), 'e')}")
            print(f"v_BC_1 loss (for one batch) at step {step}: {format(float(loss_v_BC_1), 'e')}")
            print(f"v_BC_2 loss (for one batch) at step {step}: {format(float(loss_v_BC_2), 'e')}")
            print(f"u_IV_1 loss (for one batch) at step {step}: {format(float(loss_u_IV_1), 'e')}")
            print(f"u_IV_2 loss (for one batch) at step {step}: {format(float(loss_u_IV_2), 'e')}")
            print(f"v_IV_1 loss (for one batch) at step {step}: {format(float(loss_v_IV_1), 'e')}")
            print(f"v_IV_2 loss (for one batch) at step {step}: {format(float(loss_v_IV_2), 'e')}")
            print(f"Seen so far: {(step + 1) * batch_size} samples")
    
    # Stop criteria
    loss_seq.append(loss_train)  
    stopEarly = Callback_EarlyStopping(loss_seq, min_delta=1e-02, patience=1000)
    if stopEarly or loss_train < 1e-04:
        print("Callback_EarlyStopping signal received at epoch= %d/%d"%(epoch,epochs))
        print("Terminating training ")
        break
        
    
    print(f"Time taken: {round(time.time() - start_time_inner, 2)}s")
print(100*'=')
print(f"Total time taken: {round(time.time() - start_time_outer, 2)}s")

In [ ]:
h_hat = model.predict([x_values, time_test*np.ones(N_test)])
h_hat.shape

In [ ]:
u_1_hat = h_hat[:, 0:1]
v_1_hat = h_hat[:, 1:2]
u_2_hat = h_hat[:, 2:3]
v_2_hat = h_hat[:, 3:4]

In [ ]:
N_test = 50

In [ ]:
x_values = np.linspace(0, 1, N_test)
time_test = 0.0

In [ ]:
def fun(x,t,n, fn, fmn, gn, gmn, an,bn, Cn1, Cn3, Dn1, Dn3, etn, ex1n, ex2n, psi1, psi2):
    fn = (2*z*(cmath.exp(2*z*np.pi*n)-1)*(3 - np.pi**2*n**2) + 6*np.pi*n*(cmath.exp(2*z*np.pi*n)+1))/24*np.pi**3*n**3
    fmn = (2*z*(cmath.exp(-2*z*np.pi*n)-1)*(np.pi**2*n**2-3) + 6*np.pi*n*(cmath.exp(-2*z*np.pi*n)+1))/24*np.pi**3*n**3
    gn = ((cmath.exp(2*z*np.pi*n)-1)*z*np.pi*n - 2*(cmath.exp(2*z*np.pi*n)+1) + 4*cmath.exp(z*np.pi*n))/8*np.pi**2*n**2
    gmn = (-(cmath.exp(-2*z*np.pi*n)+1)*z*np.pi*n - 2*(cmath.exp(-2*z*np.pi*n)+1) + 4*cmath.exp(z*np.pi*n))/8*np.pi**2*n**2
    an = math.sqrt(math.sqrt(4*np.pi**2*n**2 + 1)-1)/math.sqrt(math.sqrt(4*np.pi**2*n**2 - 1)+1)
    bn = math.sqrt(math.sqrt(4*np.pi**2*n**2 + 1)+1)/math.sqrt(math.sqrt(4*np.pi**2*n**2 - 1)-1)
    Cn1 = (gn + fn*bn)/(an + bn)
    Cn3 = (gn + fn*an)/(an + bn)
    Dn1 = (gmn + fmn*bn)/(an+bn)
    Dn3 = (gmn + fmn*an)/(an+bn)
    etn = cmath.exp(z*math.sqrt(4*np.pi**2*n**2 + 1)*t)
    ex1n = cmath.exp(2*z*np.pi*n*x)
    ex2n = cmath.exp(-2*z*np.pi*n*x)
    psi1 = (Cn1 + Cn3)*ex1n*etn + (Dn1 + Dn3)*ex2n*etn
    psi2 = (-an*Cn1 + bn*Cn3)*ex1n*etn + (an*Dn1 - bn*Dn3)*ex2n*etn
    return fn, fmn, gn, gmn, an,bn, Cn1, Cn3, Dn1, Dn3, etn, ex1n, ex2n, psi1, psi2

In [ ]:
fig = plt.figure()

im = plt.figure(figsize=(15,7))
plt.plot(x_values, psi_1(x_values,time_test), '-', label='Analytical solution')
plt.plot(x_values, u_1_hat, '-o', label='Neural network prediction')
plt.xlabel('$\\xi$')
plt.ylabel('u_1($\\xi,\\tau$)')
plt.ylim(-4, 4)
plt.title('Exact solution')
plt.title('Example - 1, $\\omega_{1} = \ldots = \omega_{4} = 1000000$,$\\omega_{5} = \ldots = \omega_{12} = 1$')
plt.legend()
plt.grid()